In [6]:
import numpy as np
import os
import glob
import pickle
import soundfile , librosa
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [7]:
def feature_extractor(file, mfcc, chroma, mel):
        with soundfile.SoundFile(file) as sound_file:
            file_array = sound_file.read(dtype="float32")
            sample_rate=sound_file.samplerate
            result=np.array([])
            if mfcc:
                mfccs=np.mean(librosa.feature.mfcc(y=file_array, sr=sample_rate, n_mfcc=40).T, axis=0)
                result=np.hstack((result, mfccs))
            if chroma:
                stft=np.abs(librosa.stft(file_array))
                chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
                result=np.hstack((result, chroma))
            if mel:
                mel=np.mean(librosa.feature.melspectrogram(file_array, sr=sample_rate).T,axis=0)
                result=np.hstack((result, mel))
        return result

In [8]:
emotions={
      '01':'neutral',
      '02':'calm',
      '03':'happy',
      '04':'sad',
      '05':'angry',
      '06':'fearful',
      '07':'disgust',
      '08':'surprised'
    }
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [11]:
def data_loader():
        x , y = [] , []
        for file in glob.glob("voice_dataset\Actor_*\\*.wav"):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]]
            if emotion not in observed_emotions:
                continue
            feature = feature_extractor(file, mfcc=True, chroma=True, mel=True)
            x.append(feature)
            y.append(emotion)
        return train_test_split(x, y, test_size=0.25,random_state=9)

In [12]:
x_train,x_test,y_train,y_test=data_loader()

In [143]:
classifier=MLPClassifier(alpha=0.001, batch_size=400, epsilon=1e-09, hidden_layer_sizes=(600,), learning_rate='adaptive', max_iter=600)

In [144]:
classifier.fit(x_train,y_train)

C:\Users\ACA-Ahmed g\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.001, batch_size=400, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-09,
              hidden_layer_sizes=(600,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=600, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [145]:
y_pred=classifier.predict(x_test)

In [146]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [147]:
print(accuracy)

0.8020833333333334
